#**Import Modules**

In [0]:
from google.colab import files

In [0]:
import openpyxl

In [0]:
import time

In [0]:
from os import listdir
from os.path import isfile, join

In [20]:
!pip install inflect

In [0]:
import inflect as inf

#**Mount Google Drive**

In [22]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [23]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/Paper/Trade'
/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [24]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/


#**Modify FIles**

In [0]:
DATAPATH = "./Data/raw/Korean/"

In [0]:
# fetch file names
raw_files = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]

In [0]:
engine = inf.engine()

In [0]:
def refine(kwd):
    # upper to lower
    kwd = kwd.lower()

    # plural to singular ... quite erroneous
    if engine.singular_noun(kwd):
        kwd = engine.singular_noun(kwd)

    # delete and in front
    try:
        if(kwd.index("and") == 0):
            kwd = kwd[3:]
    except ValueError:
        pass

    # delete -
    kwd = kwd.replace('-', ' ')

    # convert & to and
    kwd = kwd.replace('&', " and ")

    # strip . , ;
    kwd = kwd.strip(" .,;")

    # convert double spaces to single space
    kwd = kwd.replace("  ", ' ')

    # delete parenthesis
    try:
        while(kwd.index("(") < kwd.index(")")):
            if(kwd.index(")") == len(kwd)):
                kwd = kwd[:kwd.index("(")]
            else:
                kwd = kwd[:kwd.index("(")] + kwd[kwd.index(")") + 1:]
    except ValueError:
        pass
    
    # split by , ;
    kwd = kwd.replace(',', ';')
    kwd_split = kwd.split(';')
    
    res = []
    if(len(kwd_split) == 1):
        res.append(kwd)
    else:
        for split in kwd_split:
            res = res + refine(split)
    
    return res

In [34]:
for raw in raw_files:
    # read file
    print("Started reading, " + raw)

    # open file
    wb = openpyxl.load_workbook(DATAPATH + raw)
    if("kwd" not in wb.sheetnames):
        print(raw + " is not yet in format")
        continue
    ws_kwd = wb["kwd"]
    MAXROW = ws_kwd.max_row

    titles = []
    years = []
    journals = []
    keywords = []

    # read and refine data from kwd sheet
    for r in ws_kwd.rows:
        # ignore header
        if(r[0].row == 1):
            continue
        # halt loop
        if(r[0].row > MAXROW):
            break

        # read title
        if(isinstance(r[0].value, str)):
            titles.append(r[0].value)
        # read journal
        if(isinstance(r[1].value, str)):
            journals.append(r[1].value)
        # read year
        if(isinstance(r[2].value, str)):
            years.append(r[2].value)
        # read keywords
        kwds = []
        idx = 3
        while(idx < len(r) and isinstance(r[idx].value, str)):
            kwds.append(r[idx].value)
            idx += 1
        # refine keywords
        kwds_refined = []
        for kwd in kwds:
            kwds_refined = kwds_refined + refine(kwd)
        while ' ' in kwds_refined:
            kwds_refined.remove(' ')
        while '' in kwds_refined:
            kwds_refined.remove('')
        keywords.append(kwds_refined)
    
    print("Finished reading : " + raw)

    # write file
    print("Started writing, " + raw)
    
    # create sheet
    if("kwd_refined" in wb.sheetnames):
        wb.remove_sheet(wb["kwd_refined"])
    wb.create_sheet("kwd_refined")
    ws_kwd_refined = wb["kwd_refined"]

    # write header
    ws_kwd_refined.cell(row = 1, column = 1).value = 'title'
    ws_kwd_refined.cell(row = 1, column = 2).value = 'journal'
    ws_kwd_refined.cell(row = 1, column = 3).value = 'year'
    ws_kwd_refined.cell(row = 1, column = 4).value = 'keywords'

    # write to kwd_refined sheet
    r = 2
    for i in range(len(titles)):
        # write data to kwd_refined sheet
        ws_kwd_refined.cell(row = r, column = 1).value = titles[i]
        ws_kwd_refined.cell(row = r, column = 2).value = journals[i]
        ws_kwd_refined.cell(row = r, column = 3).value = years[i]
        idx = 4
        for kwd in keywords[i]:
            ws_kwd_refined.cell(row = r, column = idx).value = kwd
            idx += 1
        r += 1

    # save file
    wb.save(DATAPATH + raw)
    time.sleep(10)
    wb.close()

    print("Finished modifying : " + raw)

Started reading, journal_of_korea_trade.xlsx
Finished reading : journal_of_korea_trade.xlsx
Started writing, journal_of_korea_trade.xlsx


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


Finished modifying : journal_of_korea_trade.xlsx
Started reading, journal_of_international_trade_and_industry_studies.xlsx
Finished reading : journal_of_international_trade_and_industry_studies.xlsx
Started writing, journal_of_international_trade_and_industry_studies.xlsx


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


Finished modifying : journal_of_international_trade_and_industry_studies.xlsx
Started reading, korea_trade_review.xlsx
Finished reading : korea_trade_review.xlsx
Started writing, korea_trade_review.xlsx


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


Finished modifying : korea_trade_review.xlsx
